# Lesson 5: Bandwidth

Welcome to Lesson 5!

To access the `requirements.txt` and `utils5.py` file for this course, go to `File` and click `Open`.

#### 1. Load imports

In [1]:
from flwr.client.mod import parameters_size_mod

from utils5 import *

#### 2.  Define the model

*  Initialize the model.

In [2]:
model = AutoModelForCausalLM.from_pretrained(
    "EleutherAI/pythia-14m",
    cache_dir="./pythia-14m/cache",
)

config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

c:\Users\Ridoy\anaconda3\envs\flower_env\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ridoy\Federated Learning\pythia-14m\cache\models--EleutherAI--pythia-14m. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back

model.safetensors:   0%|          | 0.00/53.3M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Find more information about [EleutherAI/pythia-14m](https://huggingface.co/EleutherAI/pythia-14m)

* Get some Model values.

In [3]:
vals = model.state_dict().values()
total_size_bytes = sum(p.element_size() * p.numel() for p in vals)
total_size_mb = int(total_size_bytes / (1024**2))

log(INFO, "Model size is: {} MB".format(total_size_mb))

INFO : Model size is: 53 MB


* Define the FlowerClient.

In [4]:
class FlowerClient(NumPyClient):
    def __init__(self, net):
        self.net = net

    def fit(self, parameters, config):
        set_weights(self.net, parameters)
        # No actual training here
        return get_weights(self.net), 1, {}

    def evaluate(self, parameters, config):
        set_weights(self.net, parameters)
        # No actual evaluation here
        return float(0), int(1), {"accuracy": 0}


def client_fn(context: Context) -> FlowerClient:
    return FlowerClient(model).to_client()


client = ClientApp(
    client_fn,
    mods=[parameters_size_mod],
)

* Define the custom strategy: BandwidthTrackingFedAvg.

In [5]:
bandwidth_sizes = []


class BandwidthTrackingFedAvg(FedAvg):
    def aggregate_fit(self, server_round, results, failures):
        if not results:
            return None, {}

        # Track sizes of models received
        for _, res in results:
            ndas = parameters_to_ndarrays(res.parameters)
            size = int(sum(n.nbytes for n in ndas) / (1024**2))
            log(INFO, f"Server receiving model size: {size} MB")
            bandwidth_sizes.append(size)

        # Call FedAvg for actual aggregation
        return super().aggregate_fit(server_round, results, failures)

    def configure_fit(self, server_round, parameters, client_manager):
        # Call FedAvg for actual configuration
        instructions = super().configure_fit(
            server_round, parameters, client_manager
        )

        # Track sizes of models to be sent
        for _, ins in instructions:
            ndas = parameters_to_ndarrays(ins.parameters)
            size = int(sum(n.nbytes for n in ndas) / (1024**2))
            log(INFO, f"Server sending model size: {size} MB")
            bandwidth_sizes.append(size)

        return instructions

In [6]:
params = ndarrays_to_parameters(get_weights(model))

def server_fn(context: Context):
    strategy = BandwidthTrackingFedAvg(
        fraction_evaluate=0.0,
        initial_parameters=params,
    )
    config = ServerConfig(num_rounds=1)
    return ServerAppComponents(
        strategy=strategy,
        config=config,
    )


server = ServerApp(server_fn=server_fn)

* Run the simulation.

In [11]:
run_simulation(server_app=server,
               client_app=client,
               num_supernodes=2,
               backend_config=backend_setup
               )

INFO : Starting Flower ServerApp, config: num_rounds=1, no round_timeout
INFO : 
INFO : [INIT]
INFO : Using initial global parameters provided by strategy
INFO : Evaluating initial global parameters
INFO : 
INFO : [ROUND 1]
INFO : Server sending model size: 53 MB
INFO : Server sending model size: 53 MB
INFO : configure_fit: strategy sampled 2 clients (out of 2)
(ClientAppActor pid=22716) No module named 'utils5'
(ClientAppActor pid=22716) Traceback (most recent call last):
(ClientAppActor pid=22716)   File "c:\Users\Ridoy\anaconda3\envs\flower_env\lib\site-packages\ray\_private\serialization.py", line 387, in deserialize_objects
(ClientAppActor pid=22716)     obj = self._deserialize_object(data, metadata, object_ref)
(ClientAppActor pid=22716)   File "c:\Users\Ridoy\anaconda3\envs\flower_env\lib\site-packages\ray\_private\serialization.py", line 268, in _deserialize_object
(ClientAppActor pid=22716)     return self._deserialize_msgpack_data(data, metadata_fields)
(ClientAppActor pid=22

* Log how much bandwidth was used!

In [12]:
log(INFO, "Total bandwidth used: {} MB".format(sum(bandwidth_sizes)))

INFO : Total bandwidth used: 318 MB
